In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas as pd
import pandas_datareader.data as web
import pandas_ta
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import datetime as dt
import yfinance as yf 
import warnings
warnings.filterwarnings('ignore')

### Consulta Lista Empresas Ibov

In [ ]:
# Adicionando lista de empresas listadas na Ibov
ibov = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa')[0]
# Deixando no padrão do Yfinance
ibov['Código'] = ibov['Código'] + '.SA'
codigos = ibov['Código'].unique().tolist()

### Pegando Dados Yfinance

In [ ]:
fim = '2023-12-29'
# Pegando Dados dos últimos 5 Anos
inicio = pd.to_datetime(fim)-pd.DateOffset(365*5)
# Salvando Dados no dataframe
df = yf.download(tickers=codigos, start=inicio, end=fim).stack()
df.index.names = ['Data', 'Ticker']
df.columns = df.columns.str.lower()

### Cálculando Indicadores

* Garman-Klass Vol

##### Garman-Klass Volatility é uma medida de volatilidade que leva em consideração os preços mais altos, mais baixos, de abertura e de fechamento, e é especialmente utilizada em análises de dados intradiários no mercado financeiro.

$$
\text{Garman-Klass Volatility} = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} \left( \ln\left(\frac{H_i}{L_i}\right) \right)^2 - \frac{2}{N-1} \sum_{i=1}^{N} \left( \ln\left(\frac{C_i}{O_i}\right) \right)^2}
$$


onde:
- \(N\) é o número de períodos;
- \(H_i\) é o preço mais alto durante o período \(i\);
- \(L_i\) é o preço mais baixo durante o período \(i\);
- \(C_i\) é o preço de fechamento durante o período \(i\);
- \(O_i\) é o preço de abertura durante o período \(i\).


In [29]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)
df

adj close      close       high        low       open  \
Data       Ticker                                                            
2019-01-02 ABEV3.SA  13.257461  16.150000  16.299999  15.400000  15.400000   
           AZUL4.SA  36.330002  36.330002  36.549999  35.450001  35.970001   
           B3SA3.SA   7.830419   9.263333   9.303333   8.893333   8.950000   
           BBAS3.SA  35.497112  48.599998  49.700001  46.090000  46.200001   
           BBDC3.SA  17.396151  22.395441  22.608315  21.005508  21.274731   
...                        ...        ...        ...        ...        ...   
2023-12-28 UGPA3.SA  26.510000  26.510000  26.590000  26.320000  26.469999   
           USIM5.SA   9.290000   9.290000   9.320000   9.200000   9.300000   
           VALE3.SA  77.199997  77.199997  77.459999  76.750000  77.000000   
           WEGE3.SA  36.910000  36.910000  37.220001  36.709999  36.840000   
           YDUQ3.SA  22.420000  22.420000  22.990000  22.320000  22.719999   

                         volume  garman_klass_vol  
Data       Ticker                                  
2019-01-02 ABEV3.SA  18692900.0         -0.007056  
           AZUL4.SA   2795000.0          0.000429  
           B3SA3.SA  73845000.0         -0.005883  
           BBAS3.SA  14905300.0         -0.023984  
           BBDC3.SA   5374737.0         -0.012945  
...                         ...               ...  
2023-12-28 UGPA3.SA   3931100.0          0.000051  
           USIM5.SA   7530100.0          0.000084  
           VALE3.SA  16545800.0          0.000040  
           WEGE3.SA   3536300.0          0.000094  
           YDUQ3.SA   1983800.0          0.000369  

[79178 rows x 7 columns]